In [20]:
from bs4 import BeautifulSoup
import requests

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.document_loaders import  WebBaseLoader
import tiktoken

diseases = [
    'Hypertension', 'Hyperlipidemia', 'Obesity', 'Diabetes', 'Cataract', 
    'Dementia', 'Rhinitis', 'Gastritis', 'Periodontal%20disease', 'Hemorrhoids',
    'Alopecia'
    ]

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

urls = []
for disease in diseases:
    for num in range(1,11):
        parent_url=f'https://www.mayoclinic.org/search/search-results?q={disease}&page={num}'
        res = requests.get(parent_url, headers=headers)
        soup = BeautifulSoup(res.content, 'html.parser')
        hrefs = [a['href'] for a in soup.find_all('a', class_='azsearchlink') if 'href' in a.attrs]
        # print(disease, hrefs)
        urls += hrefs
# len(urls)

tokenizer = tiktoken.get_encoding('cl100k_base')
def token_length(text):
    tokens = tokenizer.encode(text)
    return len(tokens)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=30,
    length_function = token_length
)

# text split
all_articles = []
for url in urls:
    docs = WebBaseLoader(url).load_and_split(text_splitter)
    all_articles+=docs
all_articles[0]



USER_AGENT environment variable not set, consider setting it to identify your requests.


Document(metadata={'source': 'https://www.mayoclinic.org/diseases-conditions/high-blood-pressure/symptoms-causes/syc-20373410', 'title': 'High blood pressure (hypertension) - Symptoms & causes - Mayo Clinic', 'language': 'en'}, page_content='High blood pressure (hypertension) - Symptoms & causes - Mayo Clinic')

In [18]:
len(urls)

110

In [21]:
all_articles[0].metadata

{'source': 'https://www.mayoclinic.org/diseases-conditions/high-blood-pressure/symptoms-causes/syc-20373410',
 'title': 'High blood pressure (hypertension) - Symptoms & causes - Mayo Clinic',
 'language': 'en'}

In [31]:
text=all_articles[3].page_content
token_length(text)

99

In [32]:
len(all_articles)

11695